In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer


In [2]:
# Cargar el dataset NUEVO
file_path = 'data_evaluacion.csv'
data = pd.read_csv(file_path)


In [3]:
# Renombrar columnas
data.columns = [
    'Edad', 'Tipo_Empleo', 'Ingresos', 'Educacion', 'Anios_Educacion',
    'Estado_Civil', 'Ocupacion', 'Relacion', 'Raza', 'Genero',
    'Ganancia_Capital', 'Perdida_Capital', 'Horas_Por_Semana', 'Pais', 'Ingreso_Anual'
]

In [4]:
# Limpiar los nombres de las columnas quitando espacios al inicio y al final
data.columns = data.columns.str.strip()

# Verificar nuevamente los nombres de las columnas
print("Columnas corregidas:", data.columns)

Columnas corregidas: Index(['Edad', 'Tipo_Empleo', 'Ingresos', 'Educacion', 'Anios_Educacion',
       'Estado_Civil', 'Ocupacion', 'Relacion', 'Raza', 'Genero',
       'Ganancia_Capital', 'Perdida_Capital', 'Horas_Por_Semana', 'Pais',
       'Ingreso_Anual'],
      dtype='object')


In [5]:
# Información básica del dataset
print(data.info())

# Resumen estadístico
print(data.describe())

# Distribución de la variable objetivo
print(data['Ingreso_Anual'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48841 entries, 0 to 48840
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Edad              48841 non-null  int64 
 1   Tipo_Empleo       48841 non-null  object
 2   Ingresos          48841 non-null  int64 
 3   Educacion         48841 non-null  object
 4   Anios_Educacion   48841 non-null  int64 
 5   Estado_Civil      48841 non-null  object
 6   Ocupacion         48841 non-null  object
 7   Relacion          48841 non-null  object
 8   Raza              48841 non-null  object
 9   Genero            48841 non-null  object
 10  Ganancia_Capital  48841 non-null  int64 
 11  Perdida_Capital   48841 non-null  int64 
 12  Horas_Por_Semana  48841 non-null  int64 
 13  Pais              48841 non-null  object
 14  Ingreso_Anual     48841 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB
None
               Edad      Ingresos  Anios_Educacion  Ga

In [6]:
# Comprobar valores nulos
print(data.isnull().sum())

# Comprobar valores únicos en cada columna
for column in data.columns:
    print(f"{column}: {data[column].nunique()} valores únicos")


Edad                0
Tipo_Empleo         0
Ingresos            0
Educacion           0
Anios_Educacion     0
Estado_Civil        0
Ocupacion           0
Relacion            0
Raza                0
Genero              0
Ganancia_Capital    0
Perdida_Capital     0
Horas_Por_Semana    0
Pais                0
Ingreso_Anual       0
dtype: int64
Edad: 74 valores únicos
Tipo_Empleo: 9 valores únicos
Ingresos: 28522 valores únicos
Educacion: 16 valores únicos
Anios_Educacion: 16 valores únicos
Estado_Civil: 7 valores únicos
Ocupacion: 15 valores únicos
Relacion: 6 valores únicos
Raza: 5 valores únicos
Genero: 2 valores únicos
Ganancia_Capital: 123 valores únicos
Perdida_Capital: 99 valores únicos
Horas_Por_Semana: 96 valores únicos
Pais: 42 valores únicos
Ingreso_Anual: 2 valores únicos


In [7]:
# Imputación de valores faltantes
imputer = SimpleImputer(strategy='most_frequent')
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Codificación de variables categóricas
label_encoders = {}
for column in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Escalado de características numéricas
scaler = StandardScaler()
numerical_cols = ['Edad', 'Ingresos', 'Anios_Educacion', 'Ganancia_Capital', 'Perdida_Capital', 'Horas_Por_Semana']
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Verificación final
print(data.head())

       Edad  Tipo_Empleo  Ingresos  Educacion  Anios_Educacion  Estado_Civil  \
0  0.828300            6 -1.249723          9         1.136539             2   
1 -0.046940            4  0.582077         11        -0.419313             0   
2  1.047111            4  0.790127          1        -1.197239             2   
3 -0.776308            4  1.471042          9         1.136539             2   
4 -0.119877            4  1.187097         12         1.525502             2   

   Ocupacion  Relacion  Raza  Genero  Ganancia_Capital  Perdida_Capital  \
0          4         0     4       1         -0.277849        -0.205082   
1          6         1     4       1         -0.277849        -0.205082   
2          6         0     2       1         -0.277849        -0.205082   
3         10         5     2       0         -0.277849        -0.205082   
4          4         5     4       0         -0.277849        -0.205082   

   Horas_Por_Semana  Pais  Ingreso_Anual  
0         -2.228234    39

In [8]:
# Separar características y la variable objetivo
X = data.drop('Ingreso_Anual', axis=1)
y = data['Ingreso_Anual']

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Función para entrenar y evaluar un modelo
def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))

# Modelos a probar
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM (Linear Kernel)': SVC(kernel='linear'),
    'SVM (RBF Kernel)': SVC(kernel='rbf'),
    'SVM (Polynomial Kernel)': SVC(kernel='poly'),
    'SVM (Sigmoid Kernel)': SVC(kernel='sigmoid'),
    'K-NN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(max_iter=1000)
}




In [ ]:
# Evaluación de cada modelo
for name, model in models.items():
    print(f"Modelo: {name}")
    train_and_evaluate(model, X_train, y_train, X_test, y_test)

In [ ]:
# Selección del mejor k para k-NN
k_range = range(1, 26)
scores = {}
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scores[k] = accuracy_score(y_test, y_pred)

# Graficar el rendimiento en función de k
plt.plot(list(scores.keys()), list(scores.values()))
plt.xlabel('Número de vecinos: k')
plt.ylabel('Accuracy')
plt.title('Selección del mejor k para k-NN')
plt.show()

# Mejor k
best_k = max(scores, key=scores.get)
print(f"Mejor k: {best_k} con accuracy: {scores[best_k]:.4f}")


In [ ]:
# Evaluar underfitting y overfitting
def evaluate_fit(model, X_train, y_train, X_test, y_test):
    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)
    print(f"Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

# Evaluar el ajuste para cada modelo
for name, model in models.items():
    print(f"Evaluación de ajuste para: {name}")
    evaluate_fit(model, X_train, y_train, X_test, y_test)


In [10]:

#el mejor modelo resulto ser las redes neuronales
best_model=MLPClassifier(max_iter=1000).fit(X_train, y_train)


In [11]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8528
              precision    recall  f1-score   support

           0       0.87      0.95      0.91     11147
           1       0.79      0.53      0.63      3506

    accuracy                           0.85     14653
   macro avg       0.83      0.74      0.77     14653
weighted avg       0.85      0.85      0.84     14653



In [12]:
import pickle
# Grabamos el modelo en el directorio
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_model, file)